In [9]:
from pprint import pprint
import pandas as pd

filename = 'nwm.csv'
df = pd.read_csv(filename)
df = df.drop(df.columns[0], axis=1)

In [10]:
df

,C1,C2,C3,C4
0,0.60,0.93,0.00,0.73
1,0.66,0.55,0.45,0.49
2,1.00,0.45,0.57,0.50
3,0.48,0.87,0.00,0.75
4,0.62,0.40,0.56,0.50
5,0.78,0.27,0.71,0.50
6,0.40,0.90,0.00,0.82
7,0.64,0.44,0.54,0.54
8,0.65,0.30,0.71,0.55
9,0.45,0.86,0.00,0.73


Pary wariantów do oceny:
- 3 i 17 - para 15
- 11 i 14 - para 16
- 0 i 11 - para 9
- 13 i 25 - para 28 
- 7 i 8 - para 36

Nasza preferencja zakłda wybór wariantów o niższej sumie wartości krytriów c3 i c4. W efekcie:

- 3 jest preferowane nad 17
- 11 jest preferowane nad 14
- 0 jest preferowane nad 11
- 13 jest preferowane nad 25
- 7 jest preferowane nad 8



In [11]:
# Import the library
from pulp import *

# Create a problem variable:
prob = LpProblem("MinimizeEpsilon", LpMaximize)

# Create problem variables:
e = LpVariable("e", lowBound=0, upBound=None)
w1 = LpVariable("w1", lowBound=0, upBound=None) # w1>=0
w2 = LpVariable("w2", lowBound=0, upBound=None) # w2>=0
w3 = LpVariable("w3", lowBound=0, upBound=None) # w3>=0
w4 = LpVariable("w4", lowBound=0, upBound=None) # w4>=0

In [12]:
prob += e, "Objective Function"

# Add constraints:
prob += w1 + w2 + w3 + w4 == 1, "Sum of weights"

prob += w1 * df.iat[3, 0] + w2 * df.iat[3, 1] + w3 * df.iat[3, 2] + w4 * df.iat[3, 3] - w1 * df.iat[17, 0] - w2 * df.iat[17, 1] - w3 * df.iat[17, 2] - w4 * df.iat[17, 3] >= e, "Constraint 1"
prob += w1 * df.iat[11, 0] + w2 * df.iat[11, 1] + w3 * df.iat[11, 2] + w4 * df.iat[11, 3] - w1 * df.iat[14, 0] - w2 * df.iat[14, 1] - w3 * df.iat[14, 2] - w4 * df.iat[14, 3] >= e, "Constraint 2"
prob += w1 * df.iat[0, 0] + w2 * df.iat[0, 1] + w3 * df.iat[0, 2] + w4 * df.iat[0, 3] - w1 * df.iat[11, 0] - w2 * df.iat[11, 1] - w3 * df.iat[11, 2] - w4 * df.iat[11, 3] >= e, "Constraint 3"
prob += w1 * df.iat[13, 0] + w2 * df.iat[13, 1] + w3 * df.iat[13, 2] + w4 * df.iat[13, 3] - w1 * df.iat[25, 0] - w2 * df.iat[25, 1] - w3 * df.iat[25, 2] - w4 * df.iat[25, 3] >= e, "Constraint 4"
prob += w1 * df.iat[7, 0] + w2 * df.iat[7, 1] + w3 * df.iat[7, 2] + w4 * df.iat[7, 3] - w1 * df.iat[8, 0] - w2 * df.iat[8, 1] - w3 * df.iat[8, 2] - w4 * df.iat[8, 3] >= e, "Constraint 5"


In [13]:
prob

MinimizeEpsilon:
MAXIMIZE
1*e + 0
SUBJECT TO
Sum_of_weights: w1 + w2 + w3 + w4 = 1

Constraint_1: - e - 0.28 w1 + 0.81 w2 - w3 + 0.15 w4 >= 0

Constraint_2: - e - 0.13 w1 + 0.22 w2 - 0.2 w3 - 0.12 w4 >= 0

Constraint_3: - e - 0.14 w1 + 0.68 w2 - 0.8 w3 + 0.24 w4 >= 0

Constraint_4: - e - 0.02 w1 + 0.24 w2 - 0.32 w3 - 0.06 w4 >= 0

Constraint_5: - e - 0.01 w1 + 0.14 w2 - 0.17 w3 - 0.01 w4 >= 0

VARIABLES
e Continuous
w1 Continuous
w2 Continuous
w3 Continuous
w4 Continuous

In [14]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal production amount of x and y
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Print the optimal profit
print("Total profit is: ", value(prob.objective))

Status: Optimal
e = 0.14
w1 = 0.0
w2 = 1.0
w3 = 0.0
w4 = 0.0
Total profit is:  0.14


In [15]:
df_sorted = df.sort_values(by='C2', ascending=False)
df_sorted

,C1,C2,C3,C4
24,0.34,1.00,1.00,1.00
12,0.48,0.97,0.00,0.91
15,0.44,0.95,0.00,0.90
0,0.60,0.93,0.00,0.73
18,0.35,0.91,0.00,0.98
6,0.40,0.90,0.00,0.82
3,0.48,0.87,0.00,0.75
9,0.45,0.86,0.00,0.73
21,0.32,0.83,0.00,0.94
1,0.66,0.55,0.45,0.49
